In [ ]:
#| default_exp json_explorer

# json_explorer

> exploring the google takeout location history json.

In [ ]:
#| hide
from nbdev.showdoc import *

Imports

In [ ]:
#| export
from datetime import datetime
from collections import namedtuple
import json

In [ ]:
#| export

# make a datastructure to hold location data
Location = namedtuple("Location", ["timestamp", "latitude", "longitude", "accuracy"])

This is kinda neat that I can fire markdown in here. 
BTW #| export is magic to mean export it into the source file when we run `nbdev_export`

In [ ]:
#| export

def get_timestamp(timestamp): # Example: 2022-06-24T19:38:55.633Z or 2022-06-24T19:38:55Z
    """
    Google stores the timestamp in different formats, making this annoying
    """
    try:
        first_format = "%Y-%m-%dT%H:%M:%S.%fZ"
        timestamp=datetime.strptime(timestamp, first_format)
    except:
        second_format = "%Y-%m-%dT%H:%M:%SZ"
        timestamp=datetime.strptime(timestamp, second_format)
    return int(timestamp.timestamp())

In [ ]:
show_doc(get_timestamp)

---

#### get_timestamp

>      get_timestamp (timestamp)

Google stores the timestamp in different formats, making this annoying

|    | **Details** |
| -- | ----------- |
| timestamp | Example: 2022-06-24T19:38:55.633Z or 2022-06-24T19:38:55Z |

In [ ]:
# test data
test_string = "2022-06-24T19:38:55.633Z"
get_timestamp(test_string)

1656110335

In [ ]:
test_string2 = "2022-06-24T19:38:55Z"
get_timestamp(test_string2)

1656110335

In [ ]:
file_to_open = "sample-data/sample.json"

sample data to validate the google json export format

In [ ]:
#| export

def get_locations(file_to_open): 
    with open(file_to_open, 'r') as f:
        json_file = json.load(f)
        return [l for l in json_file["locations"]]    

In [ ]:
locations = get_locations(file_to_open)

In [ ]:
len(locations)

58

In [ ]:
locations[1]

{'latitudeE7': 446549213,
 'longitudeE7': -635836178,
 'accuracy': 45,
 'source': 'WIFI',
 'timestamp': '2016-06-29T13:13:01.899Z'}

In [ ]:
locations[-1]

{'latitudeE7': 446551683,
 'longitudeE7': -635835479,
 'accuracy': 111,
 'source': 'WIFI',
 'timestamp': '2016-06-29T16:29:08.374Z'}

In [ ]:
#| export

# build up the location history
# due to the malformed google data, not all entries have latitudeE7/longitudeE7
# and the differing timestamps, best effort to parse those.

def build_location_history(locations):
    location_history = [Location(get_timestamp(l["timestamp"]), l["latitudeE7"], l["longitudeE7"], l["accuracy"]) for l in locations if l.get("latitudeE7")]
    # sort by timestamp
    location_history.sort(key=lambda location: location.timestamp)
    return location_history

more validation of output, note that these don't get exported to the library

In [ ]:
location_history = build_location_history(locations)
assert len(location_history) == 58

In [ ]:
location_history[0]

Location(timestamp=1467216494, latitude=446549411, longitude=-635836042, accuracy=41)

In [ ]:
location_history[-1]

Location(timestamp=1467228548, latitude=446551683, longitude=-635835479, accuracy=111)

In [ ]:
from nbdev.doclinks import nbdev_export
nbdev_export()

/home/j/.cache/pypoetry/virtualenvs/regps-OZEkVZSd-py3.10/lib/python3.10/site-packages/nbdev/export.py:52: UserWarning: Could not find `#|default_exp` cell. Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn("Could not find `#|default_exp` cell. Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.\n"
